## **Tratamiento de Variables Categóricas**

Una variable categótica es aquella que representa grupos o categorías de un dataset en lugar de valores numéricos continuos, por ejemplo:

  - Color de ojos (azul, verde, café) - (nominal, no tiene orden)
  - Nivel de inglés (bajo, medio, alto) - (ordinal, cumple con una jerarquía)

Cuando trabajas con modelos de regresión (y muchos otros modelos de machine learning), las variables categóricas requieren un tratamiento especial antes de poder incluirlas. La mayoría de los algoritmos esperan entradas numéricas. Estas son las técnicas de codificación más comunes:

### **1. Codificación "One-Hot" (One-Hot Encoding)**

#### 📌 **¿Cómo funciona?**
Crea nuevas columnas binarias (0 o 1) para cada categoría única presente en la variable categórica original. Cada nueva columna representa una **categoría específica**. Si una observación pertenece a esa categoría, la columna correspondiente tendrá un valor de 1; de lo contrario (y en todas las demás columnas nuevas generadas para esa variable original), tendrá un valor de 0.

💡 **Ejemplo**: Si tienes una variable "Color" con categorías "Rojo", "Azul" y "Verde":

* Se crearían tres nuevas columnas: Color_Rojo, Color_Azul, Color_Verde.

  * Una observación con Color **Rojo** tendría:
    - Color_Rojo = 1,
    - Color_Azul = 0,
    - Color_Verde = 0.

  * Una observación con Color **Azul** tendría:
   - Color_Rojo = 0,
   - Color_Azul = 1,
   - Color_Verde = 0.

#### 📌 **Ventajas:**

- Evita asumir un orden inexistente entre las categorías (a diferencia de asignar 1, 2, 3...).

- Es compatible con la mayoría de los modelos de regresión (y otros algoritmos).

- Fácil de implementar e interpretar.

#### 📌 **Consideraciones:**

- ⚠️ **Aumenta la dimensionalidad**: El número de columnas aumenta significativamente si la variable categórica tiene muchas categorías (alta cardinalidad), lo que puede llevar a la **"maldición de la dimensionalidad"** y requerir más datos/potencia computacional.

- ⚠️ **Multicolinealidad perfecta:** Las columnas creadas son perfectamente multicolineales (una columna puede predecirse perfectamente a partir de las otras, por ejemplo, si Color_Rojo=0 y Color_Azul=0, sabes que Color_Verde debe ser 1). Para evitar problemas en algunos modelos (como regresión lineal), a menudo se elimina una de las columnas.

💡 Ejemplo en Código:

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Datos de ejemplo
data = {'Color': ['Rojo', 'Azul', 'Verde', 'Rojo', 'Azul'],
        'Valor': [10, 15, 12, 10, 15]}
df = pd.DataFrame(data)

print("DataFrame Original:")
df

DataFrame Original:


,Color,Valor
0,Rojo,10
1,Azul,15
2,Verde,12
3,Rojo,10
4,Azul,15


In [2]:
# Inicializar el OneHotEncoder
#sparse_output=False Indica que la salida debe ser un array denso (NumPy array estándar) en lugar de una matriz dispersa (sparse matrix).
# Si la cardinalidad es alta se recomienda usar una matriz dispersa (sparse matrix) porque solo guardará valores no nulos, y consume menos memoria
encoder = OneHotEncoder(sparse_output=False, dtype=int)

# Aplicar One-Hot Encoding solo a la columna 'Color'
encoded = encoder.fit_transform(df[['Color']])

# Obtener los nombres de las nuevas columnas
column_names = encoder.get_feature_names_out(['Color'])

# Crear un DataFrame con los valores codificados
df_encoded = pd.DataFrame(encoded, columns=column_names)

# Concatenar el DataFrame original sin la columna 'Color' con el codificado
df_final = pd.concat([df.drop(columns=['Color']), df_encoded], axis=1)

print("\nDataFrame con One-Hot Encoding:")
df_final


DataFrame con One-Hot Encoding:


,Valor,Color_Azul,Color_Rojo,Color_Verde
0,10,0,1,0
1,15,1,0,0
2,12,0,0,1
3,10,0,1,0
4,15,1,0,0


In [3]:
# one-hot solo con pandas

import pandas as pd

# Datos de ejemplo
data = {'Color': ['Rojo', 'Azul', 'Verde', 'Rojo', 'Azul'],
        'Valor': [10, 15, 12, 10, 15]}
df = pd.DataFrame(data)
print("DataFrame Original:")
print(df)

# Aplicar One-Hot Encoding (.get_dummies)

df_one_hot = pd.get_dummies(df, columns=['Color'], prefix='Color')
df_one_hot = df_one_hot.astype(int)

print("\nDataFrame con One-Hot Encoding:")
df_one_hot

DataFrame Original:
   Color  Valor
0   Rojo     10
1   Azul     15
2  Verde     12
3   Rojo     10
4   Azul     15

DataFrame con One-Hot Encoding:


,Valor,Color_Azul,Color_Rojo,Color_Verde
0,10,0,1,0
1,15,1,0,0
2,12,0,0,1
3,10,0,1,0
4,15,1,0,0


###**2. Codificación "Dummy" (Dummy Encoding)**

#### 📌 **¿Cómo funciona?**
Es una variante de la codificación One-Hot. Crea k-1 columnas binarias, donde k es el número de categorías únicas. Una categoría se omite y se considera la categoría de referencia o base.

💡 **Ejemplo:**
Para la variable "Color" ("Rojo", "Azul", "Verde"), se podrían crear solo dos columnas, por ejemplo, Color_Azul y Color_Verde.

- Color Rojo (categoría de referencia):
  - Color_Azul = 0,
  - Color_Verde = 0.

- Color Azul:
  - Color_Azul = 1,
  - Color_Verde = 0.

- Color Verde:
  - Color_Azul = 0,
  - Color_Verde = 1.

#### 📌 **Ventajas:**

- Evita la multicolinealidad perfecta introducida por One-Hot Encoding.

- Reduce ligeramente la dimensionalidad comparado con One-Hot (una columna menos).

- Facilita la interpretación de los coeficientes en modelos lineales: los coeficientes de las variables dummy representan la diferencia en la variable dependiente entre esa categoría y la categoría de referencia, manteniendo todo lo demás constante.

#### 📌 **Consideraciones:**

- ⚠️ La interpretación de los coeficientes depende de qué categoría se elija como referencia (la que se omite).

💡 Ejemplo en Código:

In [4]:
import pandas as pd

# Mismos datos de ejemplo
data = {'Color': ['Rojo', 'Azul', 'Verde', 'Rojo', 'Azul'],
        'Valor': [10, 15, 12, 10, 15]}
df = pd.DataFrame(data)
print("DataFrame Original:")
print(df)

# Aplicar Dummy Encoding (k-1 columnas)
# drop_first=True elimina la primera categoría ('Azul' en orden alfabético)
df_dummy = pd.get_dummies(df, columns=['Color'], prefix='Color', drop_first=True, )
df_dummy = df_dummy.astype(int)
print("\nDataFrame con Dummy Encoding:")
df_dummy
# Nota: 'Rojo' se convierte en la categoría de referencia (Color_Verde=0)
# Si quisieras 'Rojo' como referencia explícitamente, podrías reordenar categorías
# o usar otras librerías que permitan especificar la categoría a eliminar.

DataFrame Original:
   Color  Valor
0   Rojo     10
1   Azul     15
2  Verde     12
3   Rojo     10
4   Azul     15

DataFrame con Dummy Encoding:


,Valor,Color_Rojo,Color_Verde
0,10,1,0
1,15,0,0
2,12,0,1
3,10,1,0
4,15,0,0


### **3.Codificación Ordinal (Ordinal Encoding)**

#### 📌  **¿Cuándo se usa?**
Solo cuando las categorías tienen un orden intrínseco y significativo (por ejemplo, "Bajo", "Medio", "Alto"; "Pequeño", "Mediano", "Grande", etc...).

#### 📌  **¿Cómo funciona?**
Asigna un valor numérico entero a cada categoría según su orden predefinido. Por ejemplo:
- "Bajo" -> 1,
- "Medio" -> 2,
- "Alto" -> 3.

#### 📌 **Ventajas:**

- Simple y no aumenta la dimensionalidad.

- Preserva (o introduce) la información ordinal.

#### 📌 **Consideraciones:**

- ⚠️ Solo apropiada para variables verdaderamente **ordinales**. Usarla en variables nominales (sin orden, como "Color") introduce relaciones de orden falsas que confundirán al modelo.

- ⚠️ **Asume distancias iguales:** El modelo interpretará la diferencia entre 1 y 2 como igual a la diferencia entre 2 y 3. Esto puede no ser cierto en la realidad (la diferencia de impacto entre "Bajo" y "Medio" podría no ser la misma que entre "Medio" y "Alto").

💡 Ejemplo en Código (usando scikit-learn):

In [5]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

# Datos de ejemplo con orden
data = {'Talla': ['Pequeño', 'Mediano', 'Grande', 'Mediano', 'Pequeño'],
        'Valor': [5, 10, 15, 10, 5]}
df = pd.DataFrame(data)
print("DataFrame Original:")
print(df)

# Definir el orden correcto de las categorías
talla_orden = ['Pequeño', 'Mediano', 'Grande']

# Inicializar y ajustar el codificador
ordinal_encoder = OrdinalEncoder(categories=[talla_orden])
# Usamos fit_transform en la columna (requiere formato 2D)
df['Talla_Ordinal'] = ordinal_encoder.fit_transform(df[['Talla']])

print("\nDataFrame con Ordinal Encoding:")
df

DataFrame Original:
     Talla  Valor
0  Pequeño      5
1  Mediano     10
2   Grande     15
3  Mediano     10
4  Pequeño      5

DataFrame con Ordinal Encoding:


,Talla,Valor,Talla_Ordinal
0,Pequeño,5,0.0
1,Mediano,10,1.0
2,Grande,15,2.0
3,Mediano,10,1.0
4,Pequeño,5,0.0


### **4.Target Encoding ( Mean Encoding)**

#### 📌 **¿Cómo funciona?**
Reemplaza cada categoría con un **valor numérico** derivado de la variable dependiente (objetivo). Lo más común es usar la media de la variable dependiente para las observaciones que pertenecen a esa categoría. Por ejemplo, si la variable objetivo es "Precio", la categoría "Rojo" se reemplazaría por el precio promedio de todos los artículos rojos en el conjunto de entrenamiento.

#### 📌 **Ventajas:**

- Puede capturar información predictiva directamente relacionada con el objetivo.

- No aumenta significativamente la dimensionalidad (solo crea una columna).

- Muy útil en variables de alta cardinalidad donde One-Hot explotaría.

#### 📌 **Consideraciones:**

- ⚠️ **Alto riesgo de sobreajuste (Overfitting):** Si se calcula directamente sobre todo el conjunto de entrenamiento, la codificación "ve" la variable objetivo, creando una fuga de datos (data leakage). El modelo puede aprender perfectamente la relación en los datos de entrenamiento pero fallar en datos nuevos.

- ⚠️ **Necesidad de regularización/validación:** Es crucial implementar técnicas para mitigar el sobreajuste, como:

  - Calcular las medias usando solo los datos de entrenamiento (y aplicarlas al de test).

  - Usar validación cruzada interna (calcular las medias en k-1 folds y aplicarlas al fold restante).

  - Aplicar suavizado (smoothing) o ponderación, especialmente para categorías con pocas muestras.

💡 Ejemplo en Código:

In [ ]:
import pandas as pd

# Datos de ejemplo (con variable objetivo 'Precio')
data = {'Color': ['Rojo', 'Azul', 'Verde', 'Rojo', 'Azul', 'Verde', 'Rojo'],
        'Precio': [100, 150, 120, 110, 140, 130, 90]}
df = pd.DataFrame(data)
print("DataFrame Original:")
print(df)

# Calcular la media del 'Precio' para cada 'Color'
# ¡¡ADVERTENCIA: Hacer esto directamente en todo el dataset causa data leakage!! (El data leakage ocurre cuando información del conjunto de prueba se filtra durante el entrenamiento del modelo.)
# En una aplicación real, calcular esto SOLO en datos de entrenamiento
# o usar técnicas de validación cruzada.
color_mean_map = df.groupby('Color')['Precio'].mean()
print("\nMedia de Precio por Color:")
print(color_mean_map)

# Mapear la media a la columna original
df['Color_TargetEncoded'] = df['Color'].map(color_mean_map)

print("\nDataFrame con Target Encoding (conceptual):")
df

# Para una implementación más robusta, considera la librería category_encoders:
# from category_encoders import TargetEncoder
# encoder = TargetEncoder(cols=['Color'])
# df_target_encoded = encoder.fit_transform(df['Color'], df['Precio'])

DataFrame Original:
   Color  Precio
0   Rojo     100
1   Azul     150
2  Verde     120
3   Rojo     110
4   Azul     140
5  Verde     130
6   Rojo      90

Media de Precio por Color:
Color
Azul     145.0
Rojo     100.0
Verde    125.0
Name: Precio, dtype: float64

DataFrame con Target Encoding (conceptual):


,Color,Precio,Color_TargetEncoded
0,Rojo,100,100.0
1,Azul,150,145.0
2,Verde,120,125.0
3,Rojo,110,100.0
4,Azul,140,145.0
5,Verde,130,125.0
6,Rojo,90,100.0


### **5. label Encoding**

#### 📌 **¿Cómo funciona?**  
Label Encoding convierte cada categoría de una variable en un número entero único. Se asignan valores arbitrarios sin considerar ningún orden específico.  

Por ejemplo, si tenemos una variable "Color" con los valores ("Rojo", "Azul", "Verde"), podríamos asignar:  
- Azul → 0  
- Rojo → 1  
- Verde → 2  

#### 📌 **Ventajas:**  
- Es fácil de implementar y no aumenta la cantidad de columnas.  
- Útil en modelos que aceptan datos categóricos codificados numéricamente.  
- Modelos como Decision Trees y Random Forest pueden manejar bien esta codificación.  

#### 📌 **Consideraciones:**  
- ⚠️ **Relaciones numéricas erróneas:** Modelos lineales podrían interpretar que "Verde" (2) es mayor que "Rojo" (1), lo cual no tiene sentido en categorías sin orden.  
- ⚠️ **No adecuado para variables con muchas categorías:** Puede generar problemas en modelos como regresión lineal o k-means clustering.

In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Datos de ejemplo
data = {'Color': ['Rojo', 'Azul', 'Verde', 'Rojo', 'Azul'],
        'Valor': [10, 15, 12, 10, 15]}
df = pd.DataFrame(data)
print("DataFrame Original:")
print(df)

# Aplicar Label Encoding
label_encoder = LabelEncoder()
df['Color_label'] = label_encoder.fit_transform(df['Color'])

print("\nDataFrame con Label Encoding:")
print(df)

# Mostrar cómo se asignaron los valores
print("\nAsignación de valores en Label Encoding:")
for clase, valor in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)):
    print(f"{clase}: {valor}")

DataFrame Original:
   Color  Valor
0   Rojo     10
1   Azul     15
2  Verde     12
3   Rojo     10
4   Azul     15

DataFrame con Label Encoding:
   Color  Valor  Color_label
0   Rojo     10            1
1   Azul     15            0
2  Verde     12            2
3   Rojo     10            1
4   Azul     15            0

Asignación de valores en Label Encoding:
Azul: 0
Rojo: 1
Verde: 2


### **6. Codificación Binaria (Binary Encoding):**

#### 📌 **¿Cómo funciona?**  
Convierte cada categoría en un número entero y luego codifica ese entero en binario. Cada bit de la representación binaria se convierte en una nueva columna.

Se suele usar cuando existen variables categóricas con alta cardinalidad. Puede ser una alternativa a one-hot que introduce menos dimensionalidad.

#### 📌 **Ventajas:**  

- Reduce la dimensionalidad en comparación con one-hot.

#### 📌 **Consideraciones:**  
- ⚠️ La interpretación de las columnas individuales es menos directa.

In [7]:
%%capture
!pip install category_encoders

In [9]:
import pandas as pd
import category_encoders as ce

# Datos de ejemplo
data = {'Color': ['Rojo', 'Azul', 'Verde', 'Amarillo', 'Azul', 'Rojo', 'Morado', 'Ámbar', 'Topacio', 'Lila', 'Rosa', 'Salmón']}
df = pd.DataFrame(data)
print(df)

# Inicializar el codificador binario
encoder = ce.BinaryEncoder(cols=['Color'])

# Aplicar la codificación binaria
df_encoded = encoder.fit_transform(df)

# Mostrar resultado
print("DataFrame con Binary Encoding:")
df_encoded

       Color
0       Rojo
1       Azul
2      Verde
3   Amarillo
4       Azul
5       Rojo
6     Morado
7      Ámbar
8    Topacio
9       Lila
10      Rosa
11    Salmón
DataFrame con Binary Encoding:


,Color_0,Color_1,Color_2,Color_3
0,0,0,0,1
1,0,0,1,0
2,0,0,1,1
3,0,1,0,0
4,0,0,1,0
5,0,0,0,1
6,0,1,0,1
7,0,1,1,0
8,0,1,1,1
9,1,0,0,0


### **Consideraciones Generales Adicionales**

#### 📌 **Elección del Método**
La mejor técnica de codificación depende de varios factores:

- **Tipo de variable**: Si es **nominal** (sin orden) u **ordinal** (con orden).
- **Cardinalidad**: El número de categorías distintas.
- **Modelo utilizado**: Algunos modelos, como los **árboles de decisión**, manejan categorías de forma diferente y son menos sensibles a la multicolinealidad.
- **Características del dataset**: La distribución de las categorías y su relación con la variable objetivo.

#### 📌 **Evaluación**
Siempre es recomendable probar diferentes métodos de codificación y medir su impacto en el rendimiento del modelo. Para esto, puedes usar:

- **Métricas apropiadas** en un conjunto de validación.
- **Validación cruzada** para evaluar la robustez del método.

#### 📌 **Consistencia entre Train y Test**
Es fundamental aplicar la **misma codificación** a los datos de entrenamiento y a los de prueba/nuevos datos de predicción. Esto significa:

1. **Ajustar (`fit`)** el codificador solo con los datos de entrenamiento.
2. **Usar ese codificador ya ajustado** para transformar (`transform`) los datos de entrenamiento y prueba.
3. **Mantener las mismas columnas** en ambos conjuntos.

#### 📌 **Manejo de Categorías Nuevas**
Cuando en los datos de prueba aparece una categoría que **no estaba en el entrenamiento**, se deben tomar medidas para evitar errores.

✅ **Soluciones según el método de codificación:**

- **`pandas.get_dummies`**  
  - Si aplicas `pd.get_dummies(df_test, ...)` separadamente, no creará una columna para la nueva categoría.  
  - Para asegurar consistencia, puedes:  
    - **Unir `train` y `test` antes de codificar**, luego separarlos.  
    - **Usar `reindex` después de `get_dummies` en el conjunto de prueba**, asegurando que tenga las mismas columnas que `train`, llenando valores faltantes con `0`.  

- **`scikit-learn` Encoders**  
  - Tienen parámetros como `handle_unknown`:  
    - **`handle_unknown='ignore'`**: Ignora categorías nuevas (las filas tendrán ceros en columnas dummy).  
    - **`handle_unknown='error'`**: Lanza un error si hay categorías desconocidas.  

- **`OrdinalEncoder`**  
  - Puede asignar un valor especial (`-1` o `np.nan`) para categorías nuevas. Luego, es necesario manejar estos valores (por ejemplo, con imputación).  

- **`TargetEncoder`**  
  - Puede asignar un valor por defecto, como la **media global de la variable objetivo**.  

In [ ]:
# Artículo que pueden consultar: https://datos.ninja/tutorial/como-codificar-variables-categoricas/